Autores:
- Hernán Francisco Díaz Farías
- María Angélica Parra Mendoza
- Loreto Constanza Zavala Gaete

# Tarea 1
### Realice una exploración de datos para ver que datos existen, cuales de ellos entregan información relevante para el problema y cuales no, eliminación (si corresponde) de datos nulos y duplicados, entre otros.

#### Librerías a utilizar

In [141]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn import preprocessing
import matplotlib.pyplot as plt
#from kmodes.kmodes import KModes
from kmodes.kprototypes import KPrototypes

#### Cargar el archivo csv

In [142]:
# Cargar el archivo csv
df = (
    pd.read_csv(
        '../data/apartments_for_rent_classified_100K.csv',
        encoding='ISO-8859-1',
        sep=';',
        low_memory=False # Para evitar warnings de dtype
        )
    )

#### Visualización de los datos

In [143]:
df.head()

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,price_display,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
0,5668640009,housing/rent/apartment,One BR 507 & 509 Esplanade,"This unit is located at 507 & 509 Esplanade, R...",NaN,1.0,1.0,USD,No,Thumbnail,...,"$2,195",Monthly,542,507 509 Esplanade,Redondo Beach,CA,33.8520,-118.3759,RentLingo,1577360355
1,5668639818,housing/rent/apartment,Three BR 146 Lochview Drive,"This unit is located at 146 Lochview Drive, Ne...",NaN,1.5,3.0,USD,No,Thumbnail,...,"$1,250",Monthly,1500,146 Lochview Dr,Newport News,VA,37.0867,-76.4941,RentLingo,1577360340
2,5668639686,housing/rent/apartment,Three BR 3101 Morningside Drive,This unit is located at 3101 Morningside Drive...,NaN,2.0,3.0,USD,No,Thumbnail,...,"$1,395",Monthly,1650,3101 Morningside Dr,Raleigh,NC,35.8230,-78.6438,RentLingo,1577360332
3,5668639659,housing/rent/apartment,Two BR 209 Aegean Way,"This unit is located at 209 Aegean Way, Vacavi...",NaN,1.0,2.0,USD,No,Thumbnail,...,"$1,600",Monthly,820,209 Aegean Way,Vacaville,CA,38.3622,-121.9712,RentLingo,1577360330
4,5668639374,housing/rent/apartment,One BR 4805 Marquette NE,"This unit is located at 4805 Marquette NE, Alb...",NaN,1.0,1.0,USD,No,Thumbnail,...,$975,Monthly,624,4805 Marquette NE,Albuquerque,NM,35.1038,-106.6110,RentLingo,1577360308


In [144]:
df.dtypes

id                 int64
category          object
title             object
body              object
amenities         object
bathrooms        float64
bedrooms         float64
currency          object
fee               object
has_photo         object
pets_allowed      object
price            float64
price_display     object
price_type        object
square_feet        int64
address           object
cityname          object
state             object
latitude         float64
longitude        float64
source            object
time               int64
dtype: object

Los campos tipo object representan campos que pueden contener registros nulos o errores de escritura. Se revisará el campo **id** para empezar.
Creamos una función para revisar si el campo **id** es numérico o no.

In [145]:
def check_id(id):
    try:
        int(id)
        return True
    except:
        return False

numericos = df['id'].apply(check_id).sum()
no_numericos = df['id'].shape[0] - numericos
print(f'Numericos: {numericos}, No numéricos: {no_numericos}')

Numericos: 99492, No numéricos: 0


Se observa que hay 3 registros que no son numéricos. Se revisarán estos registros.

In [146]:
df[~df['id'].apply(check_id)]

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,price_display,price_type,square_feet,address,cityname,state,latitude,longitude,source,time


Se procederá a eliminar estos registros.

In [147]:
df = df[df['id'].apply(check_id)]

Se revisan registros duplicados.

In [148]:
df[df.duplicated()]

,id,category,title,body,amenities,bathrooms,bedrooms,currency,fee,has_photo,...,price_display,price_type,square_feet,address,cityname,state,latitude,longitude,source,time
41958,5508806580,housing/rent/apartment,"Las Vegas, prime location Two BR, Apartment. P...",Live in the middle of it all! South is a pet f...,"Gym,Parking,Pool",2.0,2.0,USD,No,Yes,...,"$1,335",Monthly,1084,NaN,Las Vegas,NV,36.1536,-115.1965,RentDigs.com,1568754086
41959,5508806428,housing/rent/apartment,Northglenn Value. Pet OK!,"Square footage: 738 square ft, unit number: 06...","Gym,Parking,Pool,Storage",1.0,1.0,USD,No,Yes,...,"$1,331",Monthly,738,NaN,Northglenn,CO,39.8999,-104.9442,RentDigs.com,1568754072
41960,5508806391,housing/rent/apartment,Apartment in move in condition in District Hei...,Capital Courts has something for everyone. Loc...,"AC,Parking,Playground,Pool",1.5,3.0,USD,No,Yes,...,"$1,320",Monthly,943,NaN,District Heights,MD,38.8516,-76.8871,RentDigs.com,1568754069
41961,5508806299,housing/rent/apartment,"District Heights, prime location Two BR, Apart...",Income Requirement: Must have 2. 5x the rent i...,Parking,1.0,2.0,USD,No,Yes,...,"$1,349",Monthly,931,NaN,District Heights,MD,38.8516,-76.8871,RentDigs.com,1568754061
41962,5508806233,housing/rent/apartment,"Spacious One BR, One BA","Square footage: 680 sq-ft, unit number: 122. P...","Basketball,Gym,Patio/Deck,Playground,Pool",1.0,1.0,USD,No,Yes,...,"$1,260",Monthly,680,NaN,Denver,CO,39.8999,-104.9442,RentDigs.com,1568754056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83243,5197839500,housing/rent/apartment,"Apartment, Studio, 400 sq. ft. - in a great area.",Apartment for rent in Ruckersville. Dogs Dogs ...,NaN,1.0,0.0,USD,No,No,...,$725,Monthly,400,NaN,Ruckersville,VA,38.2203,-78.3844,RentDigs.com,1550823878
83244,5197836604,housing/rent/apartment,offers the lifestyle you deserve in the center...,"Our pet friendly one, 2 and 3 beds apartment h...",NaN,2.0,2.0,USD,No,Yes,...,"$1,237",Monthly,1051,NaN,Charlotte,NC,35.3201,-80.7409,RentDigs.com,1550823780
83245,5197834189,housing/rent/apartment,"Attractive Two BR, Two BA. Pet OK!","Square footage: 1055 square feet, unit number:...","Gym,Parking,Pool",2.0,2.0,USD,No,Yes,...,"$1,476",Monthly,1055,NaN,Tempe,AZ,33.3924,-111.9265,RentDigs.com,1550823679
83246,5197828852,housing/rent/apartment,The Best of the Best in the City of Tempe! Sav...,In the heart of vibrant Tempe near Arizona Sta...,"Gym,Parking,Pool",2.0,3.0,USD,No,Yes,...,"$1,877",Monthly,1261,NaN,Tempe,AZ,33.3924,-111.9265,RentDigs.com,1550823091


Se eliminan los registros duplicados.

In [149]:
df = df.drop_duplicates()
df.duplicated().sum()

np.int64(0)

Se revisa la columna **category**.

In [150]:
df['category'].value_counts()

category
housing/rent/apartment            99347
housing/rent/commercial/retail       42
housing/rent                          7
housing/rent/short_term               4
housing/rent/home                     4
housing/rent/condo                    3
housing/rent/other                    1
Name: count, dtype: int64

Dado que la mayoría de los registros son de la categoría **Apartments**, se elegirá esta categoría para el análisis.

In [151]:
df = df[df['category'] == 'housing/rent/apartment']

Revisamos la variable **price_type**.

In [152]:
df['price_type'].value_counts()

price_type
Monthly           99343
Weekly                3
Monthly|Weekly        1
Name: count, dtype: int64

Dado que la mayoría de los registros son de tipo **monthly**, se elegirá esta categoría para el análisis.

In [153]:
df = df[df['price_type'] == 'Monthly']

Se analiza la variable **cityname**.

In [154]:
df['cityname'].value_counts()

cityname
Dallas           2856
Denver           2749
Los Angeles      2429
Las Vegas        1846
Arlington        1577
                 ... 
La Grange           1
Columbia City       1
Pomona              1
High Ridge          1
Waunakee            1
Name: count, Length: 2977, dtype: int64

Dado que la variable **cityname** tiene muchos valores, se omitirá para el análisis de clustering, ya que no se puede considerar como una variable categórica.

In [155]:
df = df.drop(columns=['cityname'])

Se revisa la variable **pets_allowed**.

In [156]:
df['pets_allowed'].value_counts()

pets_allowed
Cats,Dogs         37058
Cats               1840
Dogs                127
Cats,Dogs,None        1
Name: count, dtype: int64

Se convierte la variable **pets_allowed_cat** a tipo categórica.

In [157]:
df['pets_allowed_cat'] = df['pets_allowed'].fillna('No').map({'Cats,Dogs': 'Si', 'Cats': 'Si', 'Dogs': 'Si', 'Cats,Dogs,None': 'Si', 'No': 'No'})
df['pets_allowed_cat'].value_counts()

pets_allowed_cat
No    60317
Si    39026
Name: count, dtype: int64

In [158]:
labelencoder = preprocessing.LabelEncoder()
df['pets_allowed_cat'] = labelencoder.fit_transform(df['pets_allowed_cat'])
df['pets_allowed_cat'].value_counts()

pets_allowed_cat
0    60317
1    39026
Name: count, dtype: int64

Se revisa la **latitud** y **longitud**.

In [159]:
df[['latitude', 'longitude']]

,latitude,longitude
0,33.8520,-118.3759
1,37.0867,-76.4941
2,35.8230,-78.6438
3,38.3622,-121.9712
4,35.1038,-106.6110
...,...,...
99487,29.6151,-95.1998
99488,30.2254,-81.7579
99489,32.7379,-117.0914
99490,35.4158,-80.8451


### 2. De acuerdo con lo anterior, seleccione las variables que utilizará para realizar la segmentación, y con ello determine el modelo a utilizar.
Se seleccionarán las variables **bathrooms**, **bedrooms**, **pets_allowed_cat**, **price**, **square_feet**, **latitude** y **longitude**.
Se utilizará el modelo K-Prototypes, ya que se tienen variables numéricas y categóricas.
#TODO Falta especificar por que se descartan las variables.

In [160]:
df_vals = df.filter(items=['bathrooms','bedrooms', 'pets_allowed_cat', 'price', 'square_feet', 'latitude', 'longitude'], axis=1)

Revisamos si hay valores nulos.

In [161]:
df_vals.isnull().sum()

bathrooms            61
bedrooms            123
pets_allowed_cat      0
price                 1
square_feet           0
latitude             25
longitude            25
dtype: int64

Se eliminan los registros nulos ya que la cantidad es baja.

In [163]:
df_vals = df_vals.dropna().reset_index(drop=True)
df_vals.isnull().sum()

bathrooms           0
bedrooms            0
pets_allowed_cat    0
price               0
square_feet         0
latitude            0
longitude           0
dtype: int64

Separamos el dataframe en variables numéricas y categóricas.
**Bathrooms**, **bedrooms**, **pets_allowed_cat** son categóricas, las dos primeras no son continuas, por lo que se consideran categóricas.

In [164]:
df_vals_num = df_vals[['price','square_feet','latitude','longitude']]
df_vals_cat = df_vals[['bathrooms','bedrooms','pets_allowed_cat']]

Se estandardizan las variables numéricas y se unen con las categóricas.

In [165]:
scaler = preprocessing.StandardScaler()
df_vals_num = pd.DataFrame(scaler.fit_transform(df_vals_num), columns=df_vals_num.columns)
df_scaled = df_vals_num.join(df_vals_cat)


price               0
square_feet         0
latitude            0
longitude           0
bathrooms           0
bedrooms            0
pets_allowed_cat    0
dtype: int64